In [1]:
from docx import Document
from docx.shared import Inches

# Create a new Word document
document = Document()

# Add a table with 3 rows and 3 columns
table = document.add_table(rows=3, cols=3)

# Set the width of each column
for cell in table.columns[0].cells:
    cell.width = Inches(1.0)

for cell in table.columns[1].cells:
    cell.width = Inches(2.0)

for cell in table.columns[2].cells:
    cell.width = Inches(3.0)

# Add data to the table
table.cell(0, 0).text = 'Column 1'
table.cell(0, 1).text = 'Column 2'
table.cell(0, 2).text = 'Column 3'
table.cell(1, 0).text = 'Row 1, Column 1'
table.cell(1, 1).text = 'Row 1, Column 2'
table.cell(1, 2).text = 'Row 1, Column 3'
table.cell(2, 0).text = 'Row 2, Column 1'
table.cell(2, 1).text = 'Row 2, Column 2'
table.cell(2, 2).text = 'Row 2, Column 3'

# Save the document
document.save('my_table.docx')

ModuleNotFoundError: No module named 'docx'

In [ ]:
from docx import Document
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.text import WD_COLOR_INDEX
from docx.shared import RGBColor

# Define the arrays for mean and std values
means = [10.0, 20.0, 30.0]
stds = [1.5, 2.5, 3.5]

# Create a new Word document
document = Document()

# Add a table with headers
table = document.add_table(rows=1, cols=3)
table.alignment = WD_TABLE_ALIGNMENT.CENTER
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Condition'
hdr_cells[1].text = 'Mean ± std'
hdr_cells[2].text = 'Result'

# Add data to the table
for i in range(len(means)):
    row_cells = table.add_row().cells
    row_cells[0].text = f'Condition {i+1}'
    p = row_cells[1].paragraphs[0]
    p.add_run(f'{means[i]:.2f}').font.color.rgb = RGBColor(0, 128, 0)
    p.add_run(' ± ')
    p.add_run(f'{stds[i]:.2f}').font.color.rgb = RGBColor(255, 0, 0)
    row_cells[2].text = ''

# Save the document
document.save('my_table.docx')

In [3]:
import pandas as pd

# Define the arrays for each condition
condition1 = [1, 2, 3]
condition12 = [1, 2, 3]
condition2 = [4, 5, 6]
condition3 = [7, 8, 9]

# Create a pandas DataFrame with the arrays as columns
df = pd.DataFrame({'Condition 1': condition1 , 'Condition 2': condition2, 'Condition 3': condition3})

# Print the DataFrame
print(df)

   Condition 1  Condition 2  Condition 3
0            1            4            7
1            2            5            8
2            3            6            9


In [3]:
from prettytable import PrettyTable

# Create a prettytable object
table = PrettyTable()

# Add columns to the table
table.add_column("Name", ["Alice", "Bob", "Charlie"])
table.add_column("Age", [25, 30, 35])
table.add_column("City", ["New York", "Paris", "London"])

# Print the table
print(table)

ModuleNotFoundError: No module named 'prettytable'

In [4]:
data = [{'condition1': [1, 2, 3]},
        {'condition2': [4, 5, 6]},
        {'condition3': [7, 8, 9]}]

for d in data:
    for key, value in d.items():
        print(key, value)

condition1 [1, 2, 3]
condition2 [4, 5, 6]
condition3 [7, 8, 9]


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import pandas as pd
from docx import Document
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.shared import Cm

# Load the iris dataset
data = load_iris()
X, y = data.data, data.target

# Define the classifiers and feature extractors in an array
classifiers = [RandomForestClassifier(n_estimators=100),
               MultinomialNB(),
               LogisticRegression(),
               KNeighborsClassifier(),
               DecisionTreeClassifier(),
               SVC(kernel='linear'),
               GaussianNB()]

vectorizers = [CountVectorizer(),
               CountVectorizer(binary=True),
               TfidfVectorizer(),
               TfidfVectorizer(ngram_range=(1,2))]

# Define the scoring metrics
scoring = {'accuracy': 'accuracy',
           'precision': 'precision_macro',
           'recall': 'recall_macro',
           'f1': 'f1_macro'}

# Create a DataFrame to store the results
results = pd.DataFrame(columns=[type(vec).__name__ for vec in vectorizers])

# Fit and evaluate the models using cross-validation
for i, vec in enumerate(vectorizers):
    for j, clf in enumerate(classifiers):
        pipeline = Pipeline([('vectorizer', vec), ('classifier', clf)])
        scores = cross_val_score(pipeline, X, y, cv=5, scoring=scoring.values())
        means = [f'{score:.2f}' for score in scores.mean(axis=0)]
        stds = [f'{score:.2f}' for score in scores.std(axis=0)]
        for k, metric_name in enumerate(scoring.keys()):
            results.loc[f'{type(clf).__name__} ({metric_name})', type(vec).__name__] = f'{means[k]} ± {stds[k]}'

# Create a Word document
document = Document()

# Add a table to the document
table = document.add_table(rows=len(classifiers)+1, cols=len(vectorizers)+1)
table.style = 'Table Grid'

# Set the alignment of the table to centered
table.alignment = WD_TABLE_ALIGNMENT.CENTER

# Set the width of the first column to 3 cm
table.columns[0].width = Cm(3)

# Set the headers of the table
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Classifier / Metric'
for i, vec_name in enumerate([type(vec).__name__ for vec in vectorizers]):
    hdr_cells[i+1].text = vec_name

# Add the data to the table
for i, clf_name in enumerate([type(clf).__name__ for clf in classifiers]):
    row_cells = table.rows[i+1].cells
    row_cells[0].text = clf_name
    for j, vec_name in enumerate([type(vec).__name__ for vec in vectorizers]):
        cell_text = results.loc[f'{clf_name} ({scoring.keys()[j]})', vec_name]
        cell = row_cells[j+1]
        p = cell.paragraphs[0]
        p.add_run(cell_text.split(' ')[0]).bold = True
        p.add_run(' ± ' + cell_text.split(' ')[2])
        cell.vertical_alignment = 'center'

# Save the document
document.save('my_table.docx')

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np

# Assuming you have 4 feature extractors and 7 classifiers

# Replace these with your actual feature extractors
feature_extractors = [feature_extractor1, feature_extractor2, feature_extractor3, feature_extractor4]

# Replace these with your actual classifiers
classifiers = [classifier1, classifier2, classifier3, classifier4, classifier5, classifier6, classifier7]

# Define the metrics for evaluation
scoring_metrics = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1': make_scorer(f1_score, average='weighted')
}

# DataFrame to store results
results = pd.DataFrame(columns=['Feature Extractor', 'Classifier', 'Metric', 'Score Mean', 'Score Std'])

# Iterate over each feature extractor
for fe in feature_extractors:
    # Extract the features
    X = fe.transform(data)  # Replace 'data' with your actual data

    # Iterate over each classifier
    for clf in classifiers:
        # Iterate over each metric
        for metric_name, scorer in scoring_metrics.items():
            # Perform cross-validation
            scores = cross_val_score(clf, X, y, cv=5, scoring=scorer)  # Replace 'y' with your actual labels

            # Add the results to the DataFrame
            results = results.append({
                'Feature Extractor': fe.__class__.__name__,
                'Classifier': clf.__class__.__name__,
                'Metric': metric_name,
                'Score Mean': np.mean(scores),
                'Score Std': np.std(scores)
            }, ignore_index=True)

# Display the results
print(results)